In [25]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [26]:
from langchain_core.documents.base import Document
from typing import List, Dict, Any
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [27]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(
    "./docs.pdf",
)

In [30]:
def pdf_preprocess(docs: List[Document]):
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        encoding_name="cl100k_base",
        chunk_size=4000,
        chunk_overlap=200,
    )

    text = splitter.split_documents(docs)

    return text

In [42]:
class RAGPipeline:
    def __init__(self, file_path: str, 
                 model: str = "gpt-4o-mini", 
                 temperature: float = 0, 
                 embedding_model: str = "text-embedding-3-large",
                 from_local: str = None):
        
        self.embeddings = OpenAIEmbeddings(
                api_key=os.getenv("OPENAI_API_KEY"),
                model=embedding_model,
        )
        
        if from_local:
            self.vectorstore = FAISS.load_local(
                folder_path=from_local,
                embeddings=self.embeddings,
                allow_dangerous_deserialization=True,)
            
            self.text = self.vectorstore.documents
        else:
            self.document = PyPDFLoader(file_path).load()

            self.text = self.pdf_preprocess()

            self.model = ChatOpenAI(
                api_key=os.getenv("OPENAI_API_KEY"),
                model=model,
                temperature=temperature,
            )

            self.vectorstore = FAISS.from_documents(
                documents=self.text,
                embedding=self.embeddings,
            )

        self.prompt = hub.pull("langchain-ai/retrieval-qa-chat")

        self.combine_docs_chain = create_stuff_documents_chain(self.model, self.prompt)
        self.chain = create_retrieval_chain(self.vectorstore.as_retriever(), self.combine_docs_chain)


    def pdf_preprocess(self) -> List[Document]:
        splitter = CharacterTextSplitter.from_tiktoken_encoder(
            encoding_name="cl100k_base",
            chunk_size=4000,
            chunk_overlap=200,
        )

        splits = splitter.split_documents(self.document)

        return splits
    
    def save_vectorstore(self, path: str):
        self.vectorstore.save_local(path)
        return path

In [43]:
retriever = RAGPipeline("./docs.pdf")

In [45]:
retriever.save_vectorstore("./vectorstore")

'./vectorstore'

In [46]:
retriever.chain.invoke({"input": "Tình hình tài chính là gì?"})

{'input': 'Tình hình tài chính là gì?',
 'context': [Document(metadata={'source': './docs.pdf', 'page': 109}, page_content='Chương 3\nĐÁNH GIÁ KHÁI QUÁT TÌNH HÌNH TÀI CHÍNH\n3.1. TÌN H  HÌNH TÀ I C H ÍN H  VÀ YẼU CÀU, MỤC Đ ÍC H  ĐÁNH GIÁ \nKHÁI Q U Á T  TÌNH HÌNH TÀI CHÍNH CỦA DO A N H  NGHIỆP\n3.1.1. Tình hình tài chính\nTinh hình tài chính của doanh nghiệp thê hiện tình trạng hay thực trạng \ntài chính cùa doanh nghiệp tại một thời điêm. Tình hinh tài chính của doanh \nnghiệp tốt hay xâu, khả quan hay bi đát thể hiện rò nét chất lượng cùa toàn \nbộ hoạt động mà doanh nghiệp đã tiến hành. Nói cách khác, tình hình tài \nchính của doanh nghiệp thể hiện sự tồn tại cũng như nỗ lực của doanh \nnghiệp trên mọi mặt hoạt động, là kết quà tất yếu của mọi hoạt động mà \ndoanh nghiệp thực hiện. Dựa vào tình hình tài chính doanh nghiệp, các nhà \nquán lý biết được tình trạng tài chính hay trạng thái tài chính cụ thể cũng \nnhư xu thế phát triển của doanh nghiệp cà về an ninh tài chính, về mức độ